# IMPORTING LIBRARIES AND READING DATA

In [4]:
# Importing Modules and packages

import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from xgboost import XGBClassifier, XGBRegressor
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression

In [5]:
# Specifying constants and data paths

WINDOWS = 14
TRAIN_DATA_PATH = './Data/Data.csv'
SAMPLE_SUBMISSION_PATH = 'SampleSubmission.csv'

In [6]:
# reading data
train = pd.read_csv(TRAIN_DATA_PATH)
ss = pd.read_csv(SAMPLE_SUBMISSION_PATH)

# DATA PREPROCESSING AND FEATURE ENGINEERING

In [7]:
# delete unwanted columns
train = train.drop(['consumer_device_9','consumer_device_x'],axis=1)

In [8]:
def prep_train(ndf):
    """
    This function takes a DataFrame as input and performs preprocessing by:
    - Creating a new 'device_id' column from the 'Source' column.
    - Removing rows associated with a predefined list of unwanted device IDs.

    Parameters:
        ndf (pd.DataFrame): The input DataFrame containing at least a 'Source' column.

    Returns:
        pd.DataFrame: A cleaned DataFrame with the 'device_id' column added and 
                      rows from excluded devices removed.
    """
    df = ndf.copy()

    df['device_id'] = df['Source'].apply(lambda x: x.split('_data')[0])

    devices_to_drop = ["consumer_device_3", "consumer_device_5", "consumer_device_11",
                       "consumer_device_14", "consumer_device_15", "consumer_device_17",
                       "consumer_device_24", "consumer_device_25", "consumer_device_27",
                       "consumer_device_33", "consumer_device_4", "consumer_device_9"]

    # Filter the DataFrame to exclude rows where 'device_id' is in the 'devices_to_drop' list
    df = df[~(df['device_id'].isin(devices_to_drop))]

    return df


In [9]:
def datetime_features(ndf, datetime_column='date_time'):
    """
    This function extracts and aggregates datetime-based features from the input DataFrame.
    It adds year, month, and quarter features from a specified datetime column, 
    and optionally aggregates energy consumption (if the 'kwh' column is present).

    Parameters:
        ndf (pd.DataFrame): The input DataFrame containing at least the datetime column.
        datetime_column (str): The name of the column containing datetime information.
                               Default is 'date_time'.

    Returns:
        pd.DataFrame: A DataFrame with new datetime features added. If 'kwh' is present,
                      the data is aggregated daily per 'Source' and includes a binary 
                      column 'kwh_is_zero' to indicate zero consumption.
    """
    # Ensure the column is in datetime format
    df = pd.DataFrame()
    ndf = ndf.reset_index(drop=True)
    ndf['date'] = ndf[datetime_column].apply(lambda x: x.split(' ')[0])
    
    df[datetime_column] = pd.to_datetime(ndf[datetime_column])

    # Date features
    df[f'{datetime_column[:3]}_year'] = df[datetime_column].dt.year
    df[f'{datetime_column[:3]}_month'] = df[datetime_column].dt.month
    df[f'{datetime_column[:3]}_quarter'] = df[datetime_column].dt.quarter
    
    df = pd.concat([ndf.drop(datetime_column, axis=1).reset_index(drop=True),
                    df], axis=1)

    # Aggregate if energy consumption column is present
    if 'kwh' in df.columns:        
        df1 = df.pivot_table(index=['date', 'Source'],
                             values=['device_id', 'dat_year', 'dat_month', 'dat_quarter'],
                             aggfunc='first').reset_index()

        df2 = df.pivot_table(index=['date', 'Source'],
                             values=['kwh'],
                             aggfunc='sum').reset_index()

        df = pd.merge(df1, df2, on=['date', 'Source'], how='left')
        df['kwh_is_zero'] = np.where(df['kwh'] == 0, 1, 0)
        
    return df


In [10]:
train_prep = prep_train(train)

In [11]:
train_dt = datetime_features(train_prep)

In [12]:
def get_ts(ndf, windows=WINDOWS):
    """
    Generates lag-based features for time series modeling of energy consumption.

    For each unique 'Source' in the input DataFrame, this function computes the energy consumption
    ('kwh') from the previous `windows` days and adds them as new columns. Rows with incomplete
    lag data are dropped.

    Parameters:
        ndf (pd.DataFrame): The input DataFrame containing at least 'Source', 'date', and 'kwh' columns.
        windows (int, optional): The number of previous days to use for generating lag features.
                                 Defaults to the global variable WINDOWS.

    Returns:
        pd.DataFrame: A new DataFrame where each entry includes energy consumption values
                      from the previous `windows` days. Only rows with complete lag data are retained.
    """
    df = ndf.copy()
    new_dfs = []

    for src in df['Source'].unique():
        subset = df[df['Source'] == src].copy()
        subset['date'] = pd.to_datetime(subset['date'])
        for n in range(1, windows + 1):
            subset[f'{n}-prev-date'] = subset['date'].apply(lambda x: x - relativedelta(days=n)) 
            subset = pd.merge(
                subset,
                subset[['date', 'kwh']],
                left_on=f'{n}-prev-date',
                right_on='date',
                how='left',
                suffixes=("", f"-prev-{n}")
            )
            subset = subset.drop([f'date-prev-{n}', f'{n}-prev-date'], axis=1)

        new_dfs.append(subset.dropna())

    return pd.concat(new_dfs, axis=0).reset_index(drop=True)


In [13]:
train_ts = get_ts(train_dt)

In [14]:
def ts_fe(ndf, windows=WINDOWS):
    """
    Extracts statistical and trend-based time series features from lagged energy consumption values.

    This function computes a variety of features (e.g., mean, median, max, min, skewness, trend, etc.)
    using lagged `kwh` values for each row in the input DataFrame. It assumes that lag columns 
    (e.g., 'kwh-prev-1', ..., 'kwh-prev-N') are already present.

    Parameters:
        ndf (pd.DataFrame): The input DataFrame containing lagged `kwh` columns.
        windows (int, optional): The number of lag days to consider when extracting features.
                                 Defaults to the global variable WINDOWS.

    Returns:
        pd.DataFrame: The original DataFrame concatenated with the newly generated feature columns.
    """
    df = ndf.reset_index(drop=True).copy()

    fe_dict = {}
    ts_cols = [f'kwh-prev-{n}' for n in range(1, windows + 1)]
    ts_df = df[ts_cols].copy()

    # Statistical Features
    fe_dict['kwh-mean'] = ts_df.mean(axis=1)
    fe_dict['kwh-median'] = ts_df.median(axis=1)
    fe_dict['kwh-max'] = ts_df.max(axis=1)
    fe_dict['kwh-min'] = ts_df.min(axis=1)
    fe_dict['kwh-skew'] = ts_df.apply(lambda row: skew(row), axis=1)
    fe_dict['kwh-kurt'] = ts_df.apply(lambda row: kurtosis(row), axis=1)

    # Trend Feature
    def compute_trend(row):
        X = np.arange(len(row)).reshape(-1, 1)
        y = row.values.reshape(-1, 1)
        model = LinearRegression().fit(X, y)
        return model.coef_[0][0]

    fe_dict['kwh-trend'] = ts_df.apply(compute_trend, axis=1)

    # Differences
    diffs = ts_df.diff(axis=1).dropna(axis=1)
    fe_dict['kwh-diff-mean'] = diffs.mean(axis=1)
    fe_dict['kwh-diff-min'] = diffs.min(axis=1)

    # Rolling Statistics
    fe_dict['kwh-rolling-3mean'] = ts_df.iloc[:, :3].mean(axis=1)
    fe_dict['kwh-rolling-3std'] = ts_df.iloc[:, :3].std(axis=1)

    # Relative Features
    fe_dict['kwh-ratio-min-max'] = fe_dict['kwh-min'] / (fe_dict['kwh-max'] + 1e-6)
    fe_dict['kwh-ratio-last-first'] = ts_df.iloc[:, -1] / (ts_df.iloc[:, 0] + 1e-6)

    # Zero Count
    fe_dict['kwh-zero_count'] = windows - (ts_df == 0).sum(axis=1)

    fe_df = pd.DataFrame(fe_dict)

    return pd.concat([df, fe_df], axis=1)


In [15]:
train_fe = ts_fe(train_ts)

In [16]:
# sorting train data according date from earliest to latest
train_final = train_fe.sort_values('date')

# MODELLING

## Building Classifier

In [17]:
# specifying selected features for building classification model
sel_ind_cl = ['dat_month',
 'dat_quarter',
 'kwh-prev-1',
 'kwh-prev-2',
 'kwh-prev-8',
 'kwh-prev-14',
 'kwh-median',
 'kwh-skew',
 'kwh-kurt',
 'kwh-rolling-3std',
 'kwh-ratio-min-max',
 'kwh-zero_count']

In [18]:
# Partitioning data into features and target for classifier training
train_X = train_final[sel_ind_cl].copy()

y_cl = train_final['kwh_is_zero']


In [19]:
#specifying classifier model parameters
hyp = {'n_estimators': 898, 'max_depth': 1, 'max_leaves': 396,
       'learning_rate': 0.000842087340956581, 'gamma': 30.94607260244057,
       'min_child_weight': 3.8561108246221454, 'subsample': 0.861462043277969,
       'colsample_bytree': 0.5652677799939997, 'reg_lambda': 0.7267716296342945,
       'reg_alpha': 3.8513237586115145e-05}

In [20]:
# training and evaluating classifier
scv = StratifiedKFold(n_splits=5)
fit_models_cl = []
ll_scores = []
train_scores = []

for train_index, test_index in scv.split(train_X,y_cl):
    X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
    
    y_train, y_test = y_cl.iloc[train_index], y_cl.iloc[test_index]


    
    
    model = XGBClassifier(objective='binary:logistic',random_state=42,
                          verbosity=0,use_label_encoder=False,n_jobs=-1,
                          **hyp)

    model.fit(X_train,y_train)
    
    fit_models_cl.append(model)

    y_pred = model.predict(X_test)


    ind_cv = f1_score(y_test,y_pred)
    tr_cv = f1_score(y_train,model.predict(X_train))
    
    ll_scores.append(ind_cv)
    train_scores.append(tr_cv)

print('mean 5fold f1 train: ', np.mean(train_scores))
print('mean 5fold f1: ', np.mean(ll_scores))

mean 5fold f1 train:  0.9693272290285817
mean 5fold f1:  0.968200854168017


## Building regressor

In [21]:
# specifying features relevant for training regressor
sel_ind_reg = ['dat_quarter',
 'dat_year',
 'kwh-prev-1',
 'kwh-prev-2',
 'kwh-prev-3',
 'kwh-prev-4',
 'kwh-prev-5',
 'kwh-prev-6',
 'kwh-prev-7',
 'kwh-prev-9',
 'kwh-prev-10',
 'kwh-prev-11',
 'kwh-prev-12',
 'kwh-prev-13',
 'kwh-prev-14',
 'kwh-mean',
 'kwh-median',
 'kwh-max',
 'kwh-min',
 'kwh-trend',
 'kwh-diff-mean',
 'kwh-diff-min',
 'kwh-rolling-3mean',
 'kwh-rolling-3std',
 'kwh-ratio-min-max',
 'kwh-ratio-last-first',
 'kwh-zero_count']

In [22]:
# filtering train data to focus on non-zero consumption data for training regressor
reg_data = train_final[train_final['kwh']!=0]

#partitioning regression data into features and target
train_X = reg_data[sel_ind_reg].copy()
y = reg_data['kwh']

In [23]:
#specifying best hyperparameters for training regressor model
hyp_reg = {'n_estimators': 879, 'max_depth': 1, 'max_leaves': 315,
           'learning_rate': 0.010081865955802438, 'gamma': 97.03393880958033,
           'min_child_weight': 1.605808057056876, 'subsample': 0.9099095345150238,
           'colsample_bytree': 0.8316675562349267,
           'reg_lambda': 2.0310621003794595e-07, 'reg_alpha': 0.0020559472603671946}

In [24]:
# training and evaluating regressor
scv = KFold(n_splits=5)
fit_models_reg = []
ll_scores = []
train_scores = []

for train_index, test_index in scv.split(train_X,y):
    X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
    
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    
    model = XGBRegressor(objective='reg:squarederror',random_state=42,
                         verbosity=0,use_label_encoder=False,n_jobs=-1,
                        **hyp_reg)

    model.fit(X_train,y_train)
    
    fit_models_reg.append(model)

    y_pred = model.predict(X_test)


    ind_cv = mean_squared_error(y_test,y_pred,squared=False)
    tr_cv = mean_squared_error(y_train,model.predict(X_train),squared=False)

    
    ll_scores.append(ind_cv)
    train_scores.append(tr_cv)

print('mean 5fold rmse train: ', np.mean(train_scores))
print('mean 5fold rmse: ', np.mean(ll_scores))

mean 5fold rmse train:  3.683784255326782
mean 5fold rmse:  3.7947069607449215


# TEST DATA

## Test data preparation

In [25]:
test = ss.copy()
test['date'] = test['ID'].apply(lambda x: x.split('_')[0])
test['date_time'] = test['ID'].apply(lambda x: x.split('_')[0])
test['Source'] = test['ID'].apply(lambda x: '_'.join(x.split('_')[1:]))
test_src = test['Source'].unique()

In [26]:
ongoing = train_prep[train_prep['Source'].isin(test_src)].copy()
ongoing['date'] = ongoing['date_time'].apply(lambda x: x.split(' ')[0]) 

ongoing = ongoing.drop_duplicates(subset=['Source','date'])

date1 = ongoing.sort_values('date',ascending=True).drop_duplicates('Source',keep='last')

## Iterative test feature engineering and prediction

In [27]:
predictions = []
train_dt_ref = train_dt[train_dt['Source'].isin(test_src)]
for n in range(1,32):
    date1n = date1.copy()
    date1n['date'] = (pd.to_datetime(date1n['date_time']) + pd.Timedelta(days=n)).astype('str')
    date1n['date_time'] = date1n['date'].apply(lambda x: str(x) + ' 00:00:00')

    
    date1_dt = datetime_features(date1n)

    date1_dt['is_test'] = 1
    train_test = pd.concat([train_dt_ref,date1_dt],axis=0)
    
    traintest_ts = get_ts(train_test)
    test_ts = traintest_ts[traintest_ts['is_test'] == 1]
    test_fe = ts_fe(test_ts)

    #pred if is zero
    pred = {}
    for i,model in enumerate(fit_models_cl):
        test_prep = test_fe[sel_ind_cl] #ppipes_cl[i].transform(test_fe[sel_ind_cl])
        ktest_sel = test_prep
        y_prob = model.predict(ktest_sel)
        oof = y_prob
        pred[i] = oof    
    pred_df = pd.DataFrame(pred)
    pred_df = pred_df.mode(axis=1)
    test_fe['is_zero'] = pred_df

    #reg pred
    for i,model in enumerate(fit_models_reg):
        test_prep = test_fe[sel_ind_reg]
        ktest_sel = test_prep
        y_prob = model.predict(ktest_sel)
        oof = y_prob
        if i==0:
            preds=oof
        else:
            preds = (preds + oof)
        
        
    pred_df = pd.DataFrame(preds/5)

    test_fe['reg_pred'] = pred_df[0]

    test_fe['kwh'] = np.where(test_fe['is_zero']==1.0,0,test_fe['reg_pred'])

    predictions.append(test_fe[['date','Source','kwh']])

    train_dt_ref = pd.concat([train_dt_ref,
                               test_fe[train_dt_ref.columns].reset_index(drop=True)],
                              axis=0)

    

## Creating Submission File

In [28]:
# combining predictions for separate dates in test data into one dataframe
compiled_pred = pd.concat(predictions,axis=0)

In [29]:
# Formatting predictions dataframe to match sample submission format
compiled_pred['ID'] = compiled_pred['date'].astype('str') + '_' + compiled_pred['Source'].astype('str')
sub = compiled_pred[['ID','kwh']]

In [30]:
# exporting to submission file
sub.to_csv('ibm_skillsbuild_9th.csv',index=False)